<a href="https://colab.research.google.com/github/Kathy42xu/DL_TA/blob/main/FF5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data exploration

In [ ]:
import pandas as pd
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.\n")
except ModuleNotFoundError:
    print("Not a Colab environment, skipping Google Drive mount.\n")
except Exception as e:
    print(f"An error occurred while mounting Google Drive: {e}\n")

# --- Define file paths ---
ff5_file_path = '/content/drive/MyDrive/Fuqua/FF5.csv'
monthly_stock_file_path = '/content/drive/MyDrive/Fuqua/monthly_stock.csv'

# --- Load Data ---
print("--- Loading FF5.csv ---")
try:
    df_ff5 = pd.read_csv(ff5_file_path)
    print("FF5.csv loaded successfully.")
    print(df_ff5.head(3))
except FileNotFoundError:
    print(f"ERROR: FF5.csv not found at {ff5_file_path}")
    exit()
except Exception as e:
    print(f"ERROR reading FF5.csv: {e}")
    exit()

print("\n--- Loading monthly_stock.csv ---")
try:
    df_monthly_stock = pd.read_csv(monthly_stock_file_path)
    print("monthly_stock.csv loaded successfully.")
    print(df_monthly_stock.head(3))
except FileNotFoundError:
    print(f"ERROR: monthly_stock.csv not found at {monthly_stock_file_path}")
    exit()
except Exception as e:
    print(f"ERROR reading monthly_stock.csv: {e}")
    exit()

# --- Prepare monthly_stock Data ---
print("\n--- Preparing monthly_stock_df ---")
df_monthly_stock.columns = df_monthly_stock.columns.str.lower()


if 'mthret' in df_monthly_stock.columns:
    ret_col = 'mthret'
elif 'mthretx' in df_monthly_stock.columns:
    ret_col = 'mthretx'
    print("Using 'mthretx' as the return column since 'mthret' was not found.")
else:
    print("ERROR: Neither 'mthret' nor 'mthretx' found in monthly_stock.csv columns.")
    exit()

# Convert return column to numeric, coercing errors to NaN (CRSP specific codes)
df_monthly_stock[ret_col] = pd.to_numeric(df_monthly_stock[ret_col], errors='coerce')

# Convert 'yyyymm' to datetime period for merging
df_monthly_stock['date_period'] = pd.to_datetime(df_monthly_stock['yyyymm'], format='%Y%m').dt.to_period('M')
print(f"Selected return column: {ret_col}")
print(df_monthly_stock[['permno', 'yyyymm', 'date_period', ret_col]].head(3))

# --- Prepare FF5 Data ---
print("\n--- Preparing FF5_df ---")
df_ff5.columns = df_ff5.columns.str.lower()

# Convert 'dateff' (e.g., 19770131) to datetime period for merging
try:
    df_ff5['date_period'] = pd.to_datetime(df_ff5['dateff'], format='%Y%m%d').dt.to_period('M')
except ValueError:
    # If 'dateff' is not YYYYMMDD, try other common formats or prompt user
    print(f"Warning: Could not parse 'dateff' as YYYYMMDD. Current head of 'dateff':\n{df_ff5['dateff'].head()}")
    # Add more robust date parsing if needed, or ensure 'dateff' is consistently YYYYMMDD
    # For now, let's assume it is YYYYMMDD as per typical K. French files.
    # If it's already YYYYMM (e.g., from WRDS Fama-French), then format='%Y%m'
    if str(df_ff5['dateff'].iloc[0]).endswith('28') or str(df_ff5['dateff'].iloc[0]).endswith('29') or \
       str(df_ff5['dateff'].iloc[0]).endswith('30') or str(df_ff5['dateff'].iloc[0]).endswith('31'):
        print("Assuming 'dateff' is YYYYMMDD based on common month-end values.")
    else:
        print("Attempting to parse 'dateff' as YYYYMM.")
        df_ff5['date_period'] = pd.to_datetime(df_ff5['dateff'], format='%Y%m').dt.to_period('M')


# Ensure 'rf' (risk-free rate) is numeric and in decimal form
# Data from K. French's website is often in percent.
df_ff5['rf'] = pd.to_numeric(df_ff5['rf'], errors='coerce')
if df_ff5['rf'].abs().max() > 0.5 and df_ff5['rf'].abs().max() <=100 : # Heuristic: if max absolute value is > 0.5 (e.g. 1 for 1%), assume it's percent
    print("Risk-free rate ('rf') in FF5.csv appears to be in percentage, dividing by 100.")
    df_ff5['rf'] = df_ff5['rf'] / 100.0
else:
    print("Risk-free rate ('rf') in FF5.csv appears to be in decimal form.")

print(df_ff5[['dateff', 'date_period', 'rf']].head(3))

# --- Merge DataFrames ---
print("\n--- Merging monthly stock data with risk-free rates ---")
df_rf_to_merge = df_ff5[['date_period', 'rf']].drop_duplicates(subset=['date_period'])

df_merged = pd.merge(df_monthly_stock, df_rf_to_merge, on='date_period', how='left')
print("Merge complete.")
print(df_merged[['permno', 'yyyymm', 'date_period', ret_col, 'rf']].head(3))

# --- Calculate Excess Return ---
print("\n--- Calculating Excess Return ---")
df_merged['excess_return'] = df_merged[ret_col] - df_merged['rf']
print("Excess return calculation complete.")

# --- Display Results and Check for Missing Values ---
print("\n--- DataFrame with Excess Return (first 5 rows) ---")
print(df_merged[['permno', 'yyyymm', 'date_period', ret_col, 'rf', 'excess_return']].head())

print("\n--- Statistics for Calculated Excess Return ---")
print(df_merged['excess_return'].describe())

missing_excess_returns = df_merged['excess_return'].isnull().sum()
total_rows = len(df_merged)
print(f"\nNumber of rows with missing excess_return: {missing_excess_returns} out of {total_rows}")
if missing_excess_returns > 0:
    missing_rf_in_merged = df_merged['rf'].isnull().sum()
    missing_ret_in_merged = df_merged[ret_col].isnull().sum()
    print(f"  Breakdown: Missing 'rf' after merge: {missing_rf_in_merged}")
    print(f"             Missing '{ret_col}' (original or after coerce): {missing_ret_in_merged}")
    print("  This could be due to non-overlapping date ranges or original missing data.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.

--- Loading FF5.csv ---
FF5.csv loaded successfully.
    mktrf     smb     hml     rmw     cma      rf     umd    dateff
0 -0.0405  0.0590  0.0427 -0.0051  0.0197  0.0036  0.0400  19770131
1 -0.0194  0.0107  0.0047 -0.0016 -0.0022  0.0035  0.0036  19770228
2 -0.0137  0.0131  0.0109 -0.0030 -0.0006  0.0038  0.0055  19770331

--- Loading monthly_stock.csv ---


<ipython-input-3-802fcfccf405>:33: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_monthly_stock = pd.read_csv(monthly_stock_file_path)


monthly_stock.csv loaded successfully.
   hdrcusip    mthcaldt  permno  permco  siccd  yyyymm  mthprc    mthret  \
0  00080010  1977-01-31   10006   22156   3743  197701  35.625  0.014235   
1  00147610  1977-01-31   10014   22157   3714  197701   4.375 -0.027778   
2  00074210  1977-01-31   10050      13   3448  197701  11.000  0.173333   

    mthretx    mthvol    cusip ticker primaryexch  shrout  
0  0.014235  129300.0    80010    ACF           N  8675.0  
1 -0.027778  204400.0   147610     AJ           N  5019.0  
2  0.173333       NaN  2475610   ABLD           Q   755.0  

--- Preparing monthly_stock_df ---
Selected return column: mthret
   permno  yyyymm date_period    mthret
0   10006  197701     1977-01  0.014235
1   10014  197701     1977-01 -0.027778
2   10050  197701     1977-01  0.173333

--- Preparing FF5_df ---
Risk-free rate ('rf') in FF5.csv appears to be in decimal form.
     dateff date_period      rf
0  19770131     1977-01  0.0036
1  19770228     1977-02  0.0035
2  

#FF 5

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats.mstats import winsorize # For diagnostic winsorization
import os

# --- 1. Mount Google Drive (if files are in Drive) ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.\n")
except ModuleNotFoundError:
    print("Not a Colab environment, skipping Google Drive mount.\n")
except Exception as e:
    print(f"An error occurred while mounting Google Drive: {e}\n")
    print("Please ensure you are in a Google Colab environment and have authorized Drive access.\n")

# --- 2. Define file paths ---
ff5_file_path = '/content/drive/MyDrive/Fuqua/FF5.csv'
monthly_stock_file_path = '/content/drive/MyDrive/Fuqua/monthly_stock.csv'

# --- 3. Load and Prepare Data ---
print("--- Loading and Preparing Data ---")
try:
    df_ff5_raw = pd.read_csv(ff5_file_path)
    df_monthly_stock_raw = pd.read_csv(monthly_stock_file_path)
    print("Raw CSV files loaded.")

    # Prepare monthly_stock_df
    df_monthly_stock = df_monthly_stock_raw.copy()
    df_monthly_stock.columns = df_monthly_stock.columns.str.lower() # Ensure lowercase columns

    if 'siccd' not in df_monthly_stock.columns:
        print("WARNING: 'siccd' column not found in monthly_stock.csv. Cannot filter financial firms by SIC code.")
        df_monthly_stock['siccd'] = np.nan # Add dummy to prevent errors if used later
    else:
        df_monthly_stock['siccd'] = pd.to_numeric(df_monthly_stock['siccd'], errors='coerce')

    if 'mthret' in df_monthly_stock.columns:
        ret_col = 'mthret'
    elif 'mthretx' in df_monthly_stock.columns:
        ret_col = 'mthretx'
        print("Using 'mthretx' as the return column.")
    else:
        raise ValueError("Return column ('mthret' or 'mthretx') not found in monthly_stock.csv.")
    df_monthly_stock[ret_col] = pd.to_numeric(df_monthly_stock[ret_col], errors='coerce')
    df_monthly_stock['date_period'] = pd.to_datetime(df_monthly_stock['yyyymm'], format='%Y%m').dt.to_period('M')
    print("Monthly stock data pre-processing done.")

    # Prepare FF5_df
    df_ff5 = df_ff5_raw.copy()
    df_ff5.columns = df_ff5.columns.str.lower()
    try:
        df_ff5['date_period'] = pd.to_datetime(df_ff5['dateff'], format='%Y%m%d').dt.to_period('M')
    except ValueError:
        print("Warning: Could not parse 'dateff' as YYYYMMDD. Trying YYYYMM.")
        df_ff5['date_period'] = pd.to_datetime(df_ff5['dateff'], format='%Y%m').dt.to_period('M')

    df_ff5['rf'] = pd.to_numeric(df_ff5['rf'], errors='coerce')
    # Check if RF is in percentage (heuristic: values typically between 0 and 15 for monthly percent)
    if df_ff5['rf'].abs().max() > 0.5 and df_ff5['rf'].abs().max() < 15 : # Adjust threshold if needed
        print("Risk-free rate ('rf') in FF5.csv appears to be in percentage, dividing by 100.")
        df_ff5['rf'] = df_ff5['rf'] / 100.0
    else:
        print("Risk-free rate ('rf') in FF5.csv assumed to be in decimal form.")
    print("FF5 data pre-processing done.")

    # Merge for excess return calculation
    df_rf_to_merge = df_ff5[['date_period', 'rf']].drop_duplicates(subset=['date_period'])
    # Select necessary columns from monthly_stock to avoid duplicate columns if script re-run
    cols_to_carry = ['permno', 'date_period', ret_col, 'siccd']
    existing_cols_to_carry = [col for col in cols_to_carry if col in df_monthly_stock.columns]
    df_merged = pd.merge(df_monthly_stock[existing_cols_to_carry], df_rf_to_merge, on='date_period', how='left')
    df_merged['excess_return'] = df_merged[ret_col] - df_merged['rf']
    print("Excess return calculated.")

    # Merge with FF5 factors
    factor_columns = ['mktrf', 'smb', 'hml', 'rmw', 'cma']
    df_ff5_factors = df_ff5[['date_period'] + factor_columns].copy()
    for factor in factor_columns:
        df_ff5_factors[factor] = pd.to_numeric(df_ff5_factors[factor], errors='coerce')
        # Check if factors are in percentage
        if df_ff5_factors[factor].abs().max() > 1 and df_ff5_factors[factor].abs().max() < 50: # Heuristic for monthly factor returns
            print(f"Factor '{factor}' appears to be in percentage, dividing by 100.")
            df_ff5_factors[factor] = df_ff5_factors[factor] / 100.0
        else:
            print(f"Factor '{factor}' assumed to be in decimal form.")


    final_df_unfiltered = pd.merge(df_merged, df_ff5_factors, on='date_period', how='inner')
    final_df_unfiltered.dropna(subset=['excess_return'] + factor_columns, inplace=True)
    final_df_unfiltered['year'] = final_df_unfiltered['date_period'].dt.year
    print("Final unfiltered DataFrame created.")

except FileNotFoundError:
    print(f"ERROR: One of the CSV files not found. Please check paths:\nFF5: {ff5_file_path}\nMonthly Stock: {monthly_stock_file_path}")
    exit()
except Exception as e:
    print(f"ERROR during data loading and preparation: {e}")
    raise # Re-raise the exception to see the full traceback

# --- VERIFY DATA SCALES ---
print("\n--- Verifying Data Scales (Unfiltered Data) ---")
print("Excess Return (sample):")
print(final_df_unfiltered['excess_return'].describe())
for factor in factor_columns:
    print(f"\nFactor {factor} (sample):")
    print(final_df_unfiltered[factor].describe())

# --- Apply Common Filters ---
print("\n--- Applying Filters ---")
# Filter 1: Exclude financial firms (SIC codes 6000-6999)
if 'siccd' in final_df_unfiltered.columns:
    final_df = final_df_unfiltered[~final_df_unfiltered['siccd'].between(6000, 6999)].copy()
    print(f"Excluded financial firms (SIC 6000-6999). Rows from {len(final_df_unfiltered)} to {len(final_df)}")
else:
    final_df = final_df_unfiltered.copy()
    print("SICCD column not found, could not filter financial firms.")

print(f"Final DataFrame for analysis has {len(final_df)} rows after filtering.")


# --- 4. Define In-sample and Out-of-sample Periods ---
train_start_year = 1977
train_end_year = 2006
test_start_year = 2007
# Use the actual max year from the filtered data or 2023, whichever is earlier
data_max_year = final_df['year'].max() if not final_df.empty else train_end_year # Default to train_end_year if df is empty
test_end_year = min(data_max_year, 2023)


train_df_all_stocks = final_df[(final_df['year'] >= train_start_year) & (final_df['year'] <= train_end_year)]
test_df_all_stocks = final_df[(final_df['year'] >= test_start_year) & (final_df['year'] <= test_end_year)]

if not train_df_all_stocks.empty:
    print(f"\nTraining period (after filtering): {train_df_all_stocks['date_period'].min()} to {train_df_all_stocks['date_period'].max()}, {len(train_df_all_stocks)} observations")
else:
    print("\nTraining DataFrame is empty AFTER FILTERS.")

if not test_df_all_stocks.empty:
    print(f"Testing period (after filtering): {test_df_all_stocks['date_period'].min()} to {test_df_all_stocks['date_period'].max()}, {len(test_df_all_stocks)} observations")
else:
    print("Testing DataFrame is empty AFTER FILTERS.")

if train_df_all_stocks.empty or test_df_all_stocks.empty:
    print("ERROR: Training or testing DataFrame is empty. Check date ranges, data availability, and filters.")
    exit()

# --- 5. Perform Time-Series FF5 Regression and Collect Errors ---
in_sample_actuals_all = []
in_sample_predictions_all = []
oos_actuals_all = []
oos_predictions_all = []

firm_level_in_sample_maes = []
firm_level_oos_maes = []

# Store in-sample R-squared from individual time-series regressions
firm_level_in_sample_r_squareds = []

unique_permnos = train_df_all_stocks['permno'].unique()
min_obs_for_regression = 60

print(f"\nStarting FF5 regressions for {len(unique_permnos)} unique permnos (post-filtering)...")
processed_count = 0

for i, permno_val in enumerate(unique_permnos):
    if (i + 1) % 500 == 0: # Print progress every 500 stocks
        print(f"Processing permno {i+1}/{len(unique_permnos)}...")

    stock_train_data = train_df_all_stocks[train_df_all_stocks['permno'] == permno_val].copy()
    stock_test_data = test_df_all_stocks[test_df_all_stocks['permno'] == permno_val].copy()

    if len(stock_train_data) < min_obs_for_regression:
        continue

    Y_train = stock_train_data['excess_return']
    X_train_factors = stock_train_data[factor_columns]
    X_train_reg = sm.add_constant(X_train_factors) # Adds a 'const' column

    try:
        model = sm.OLS(Y_train, X_train_reg).fit()
        firm_level_in_sample_r_squareds.append(model.rsquared)

        # In-sample predictions and errors
        in_sample_predictions_stock = model.predict(X_train_reg)
        in_sample_actuals_all.extend(Y_train.tolist())
        in_sample_predictions_all.extend(in_sample_predictions_stock.tolist())

        # MAE Calculation (Original, without winsorization by default)
        current_in_sample_mae = mean_absolute_error(Y_train, in_sample_predictions_stock)
        firm_level_in_sample_maes.append(current_in_sample_mae)


        # Out-of-sample predictions and errors
        if not stock_test_data.empty and len(stock_test_data['excess_return']) > 0:
            Y_test_actual = stock_test_data['excess_return']
            X_test_factors = stock_test_data[factor_columns]
            X_test_reg = sm.add_constant(X_test_factors, has_constant='add')

            # Ensure X_test_reg has the same columns in the same order as X_train_reg
            # This handles cases where a factor might be all NaN in a small test set for a stock (though unlikely for FF factors)
            X_test_reg_aligned = X_test_reg.reindex(columns=X_train_reg.columns, fill_value=0)
            oos_predictions_stock = model.predict(X_test_reg_aligned)

            oos_actuals_all.extend(Y_test_actual.tolist())
            oos_predictions_all.extend(oos_predictions_stock.tolist())

            current_oos_mae = mean_absolute_error(Y_test_actual, oos_predictions_stock)
            firm_level_oos_maes.append(current_oos_mae)

        processed_count +=1
    except Exception as e:
        # print(f"Could not run regression or predict for permno {permno_val}: {e}")
        continue

print(f"Finished FF5 regressions. Successfully processed {processed_count} firms.")

# --- 6. Calculate Pooled RMSE and Average of Firm-Level MAEs ---
print("\n--- FF5 Model Performance Metrics (as per Graph.pdf Section 3.2.3 definitions) ---")

# In-sample Metrics
if in_sample_actuals_all:
    in_sample_actuals_np = np.array(in_sample_actuals_all)
    in_sample_predictions_np = np.array(in_sample_predictions_all)
    pooled_in_sample_rmse = np.sqrt(mean_squared_error(in_sample_actuals_np, in_sample_predictions_np))
    avg_firm_level_in_sample_mae = np.mean(firm_level_in_sample_maes) if firm_level_in_sample_maes else np.nan
    avg_firm_level_in_sample_r_squared = np.mean(firm_level_in_sample_r_squareds) if firm_level_in_sample_r_squareds else np.nan
    #??? definition of R^2


    print(f"\nIn-sample Pooled RMSE: {pooled_in_sample_rmse:.4f}")
    print(f"In-sample Average of Firm-Level MAEs (Graph.pdf 'MAPE'): {avg_firm_level_in_sample_mae:.4f}")
    print(f"In-sample Average of Firm-Level Time-Series R-squared: {avg_firm_level_in_sample_r_squared:.4f}") # Added this
else:
    print("\nNo in-sample results to report (no firms met regression criteria).")

# Out-of-sample Metrics
if oos_actuals_all:
    oos_actuals_np = np.array(oos_actuals_all)
    oos_predictions_np = np.array(oos_predictions_all)
    pooled_oos_rmse = np.sqrt(mean_squared_error(oos_actuals_np, oos_predictions_np))
    avg_firm_level_oos_mae = np.mean(firm_level_oos_maes) if firm_level_oos_maes else np.nan

    print(f"\nOut-of-sample Pooled RMSE: {pooled_oos_rmse:.4f}")
    print(f"Out-of-sample Average of Firm-Level MAEs (Graph.pdf 'MAPE'): {avg_firm_level_oos_mae:.4f}")

    # Calculate the Pooled Out-of-Sample R-squared (vs. zero forecast, like ML.pdf Eq 19)
    ssr_oos_pooled = np.sum((oos_actuals_np - oos_predictions_np)**2)
    sst_zero_benchmark_oos_pooled = np.sum(oos_actuals_np**2)
    if sst_zero_benchmark_oos_pooled == 0: # Avoid division by zero
        r_squared_oos_pooled_ml_style = np.nan
    else:
        r_squared_oos_pooled_ml_style = 1 - (ssr_oos_pooled / sst_zero_benchmark_oos_pooled)
    print(f"Pooled Out-of-Sample R-squared (vs. zero forecast, ML.pdf Eq 19 style): {r_squared_oos_pooled_ml_style:.4f}")

else:
    print("\nNo out-of-sample results to report (no firms had test data or met regression criteria).")

# --- Optional: Sanity check distribution of MAEs ---
if firm_level_oos_maes:
    print("\n--- Distribution of Firm-Level Out-of-Sample MAEs ---")
    print(pd.Series(firm_level_oos_maes).describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))
if firm_level_in_sample_maes:
    print("\n--- Distribution of Firm-Level In-Sample MAEs ---")
    print(pd.Series(firm_level_in_sample_maes).describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.

--- Loading and Preparing Data ---


<ipython-input-1-e3a10ceb0242>:28: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_monthly_stock_raw = pd.read_csv(monthly_stock_file_path)


Raw CSV files loaded.
Monthly stock data pre-processing done.
Risk-free rate ('rf') in FF5.csv assumed to be in decimal form.
FF5 data pre-processing done.
Excess return calculated.
Factor 'mktrf' assumed to be in decimal form.
Factor 'smb' assumed to be in decimal form.
Factor 'hml' assumed to be in decimal form.
Factor 'rmw' assumed to be in decimal form.
Factor 'cma' assumed to be in decimal form.
Final unfiltered DataFrame created.

--- Verifying Data Scales (Unfiltered Data) ---
Excess Return (sample):
count    3.726023e+06
mean     6.774702e-03
std      1.839996e-01
min     -1.012400e+00
25%     -6.520000e-02
50%     -2.200000e-03
75%      6.237300e-02
max      2.399660e+01
Name: excess_return, dtype: float64

Factor mktrf (sample):
count    3.726023e+06
mean     6.584337e-03
std      4.530174e-02
min     -2.324000e-01
25%     -1.970000e-02
50%      1.090000e-02
75%      3.560000e-02
max      1.365000e-01
Name: mktrf, dtype: float64

Factor smb (sample):
count    3.726023e+06
mea